In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("Basics").getOrCreate()

23/03/09 23:21:06 WARN Utils: Your hostname, TIGER03531 resolves to a loopback address: 127.0.1.1; using 172.24.225.156 instead (on interface eth0)
23/03/09 23:21:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/09 23:21:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("online-retail-dataset.csv", header=True)

In [4]:
df.show()

+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|   InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+--------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/2010 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/2010 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/2010 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/2010 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/2010 8:26|     4.

In [5]:
df.cache()

DataFrame[InvoiceNo: string, StockCode: string, Description: string, Quantity: string, InvoiceDate: string, UnitPrice: string, CustomerID: string, Country: string]

In [6]:
df.createOrReplaceTempView("retail_table")

### Count

In [7]:
df.count()

541909

In [8]:
from pyspark.sql.functions import count

In [9]:
df.select(count("StockCode")).show()

+----------------+
|count(StockCode)|
+----------------+
|          541909|
+----------------+



In [10]:
df.select(count("Country")).show()

+--------------+
|count(Country)|
+--------------+
|        541909|
+--------------+



In [11]:
spark.sql("select count(*) from retail_table").show()

+--------+
|count(1)|
+--------+
|  541909|
+--------+



### Count Distinct

In [12]:
from pyspark.sql.functions import countDistinct

In [13]:
df.select(countDistinct("StockCode")).show()

+-------------------------+
|count(DISTINCT StockCode)|
+-------------------------+
|                     4070|
+-------------------------+



In [14]:
df.select(countDistinct("Country")).show()

+-----------------------+
|count(DISTINCT Country)|
+-----------------------+
|                     38|
+-----------------------+



In [15]:
spark.sql("select  approx_count_distinct(StockCode, 0.05) from retail_table").show()

23/03/09 23:21:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3804|
+--------------------------------+



### First and Last

In [16]:
from pyspark.sql.functions import first, last

In [17]:
df.select(first("StockCode"), last("StockCode")).show()

+----------------+---------------+
|first(StockCode)|last(StockCode)|
+----------------+---------------+
|          85123A|          22138|
+----------------+---------------+



In [18]:
spark.sql("select first(Country), last(Country) from retail_table").show()

+--------------+-------------+
|first(Country)|last(Country)|
+--------------+-------------+
|United Kingdom|       France|
+--------------+-------------+



### Min and Max

In [19]:
from pyspark.sql.functions import min , max

In [20]:
df.select(min("UnitPrice"), max("UnitPrice")).show()

+--------------+--------------+
|min(UnitPrice)|max(UnitPrice)|
+--------------+--------------+
|     -11062.06|         99.96|
+--------------+--------------+



In [21]:
spark.sql("select min(InvoiceNo), max(InvoiceNo) from retail_table").show()

+--------------+--------------+
|min(InvoiceNo)|max(InvoiceNo)|
+--------------+--------------+
|        536365|       C581569|
+--------------+--------------+



### Sum

In [22]:
from pyspark.sql.functions import sum

In [23]:
df.select(sum("Quantity")).show()

+-------------+
|sum(Quantity)|
+-------------+
|    5176450.0|
+-------------+



In [24]:
spark.sql('select sum(Quantity) from retail_table').show()

+-------------+
|sum(Quantity)|
+-------------+
|    5176450.0|
+-------------+



### Sum Distinct

In [25]:
from pyspark.sql.functions import sumDistinct

In [26]:
df.select(sumDistinct("Quantity")).show()

/home/aryaman_sharan/miniconda3/lib/python3.9/site-packages/pyspark/sql/functions.py:316: FutureWarning: Deprecated in 3.2, use sum_distinct instead.
  warnings.warn("Deprecated in 3.2, use sum_distinct instead.", FutureWarning)


+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|               29310.0|
+----------------------+



In [27]:
spark.sql("select sum(DISTINCT Quantity) from retail_table").show()

+----------------------+
|sum(DISTINCT Quantity)|
+----------------------+
|               29310.0|
+----------------------+



### Avg

In [28]:
from pyspark.sql.functions import avg

In [29]:
df.select(avg("Quantity").alias("Avg Quantity")).show()

+----------------+
|    Avg Quantity|
+----------------+
|9.55224954743324|
+----------------+



In [30]:
spark.sql("select avg(Quantity) as Avg_Quantity from retail_table").show()

+----------------+
|    Avg_Quantity|
+----------------+
|9.55224954743324|
+----------------+



### Grouping

In [31]:
df.groupBy("InvoiceNo", "CustomerId").count().show()


+---------+----------+-----+
|InvoiceNo|CustomerId|count|
+---------+----------+-----+
|   536395|     13767|   14|
|   536609|     17850|   16|
|   536785|     15061|    6|
|   537205|     13034|   11|
|   537829|     15498|   19|
|   538513|     15454|   25|
|  C538642|     12476|    2|
|   538808|     14057|   43|
|   538830|     14298|    5|
|   539374|     14769|   22|
|   539688|     12678|    1|
|  C539956|     12980|    1|
|   539988|     18116|   89|
|  C540384|     14961|    3|
|   540498|     17243|   10|
|   541595|     14606|   20|
|   541858|     16983|   14|
|  C542218|     14606|    3|
|   536596|      null|    6|
|   537252|      null|    1|
+---------+----------+-----+
only showing top 20 rows



In [32]:
from pyspark.sql.functions import expr

In [33]:
df.groupBy("InvoiceNo").agg(expr("avg(Quantity)"),expr("stddev_pop(Quantity)"))\
.show()

+---------+------------------+--------------------+
|InvoiceNo|     avg(Quantity)|stddev_pop(Quantity)|
+---------+------------------+--------------------+
|   536596|               1.5|  1.1180339887498947|
|   536938|33.142857142857146|  20.698023172885524|
|   537252|              31.0|                 0.0|
|   537691|              8.15|   5.597097462078001|
|   538041|              30.0|                 0.0|
|   538184|12.076923076923077|   8.142590198943392|
|   538517|3.0377358490566038|  2.3946659604837897|
|   538879|21.157894736842106|  11.811070444356483|
|   539275|              26.0|  12.806248474865697|
|   539630|20.333333333333332|  10.225241100118645|
|   540499|              3.75|  2.6653642652865788|
|   540540|2.1363636363636362|  1.0572457590557278|
|  C540850|              -1.0|                 0.0|
|   540976|10.520833333333334|   6.496760677872902|
|   541432|             12.25|  10.825317547305483|
|   541518| 23.10891089108911|  20.550782784878713|
|   541783|1

### Window Functions

In [34]:
from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

In [35]:
from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window\
.partitionBy("CustomerId", "date")\
.orderBy(desc("Quantity"))\
.rowsBetween(Window.unboundedPreceding, Window.currentRow)

In [36]:
from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)


In [37]:
from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)


In [38]:
dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId").select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()


23/03/09 23:21:22 ERROR Executor: Exception in task 4.0 in stage 71.0 (TID 181)
org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '8/11/2011 12:44' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.AbstractPartialFunction.apply(AbstractPartialFunction.scala:38)
	at org.apache.spark.sql.catalyst.util.Iso8601TimestampFormatter.parse(TimestampFormatter.scala:176)
	at org.apache.spa

Py4JJavaError: An error occurred while calling o145.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 71.0 failed 1 times, most recent failure: Lost task 2.0 in stage 71.0 (TID 179) (172.24.225.156 executor driver): org.apache.spark.SparkUpgradeException: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '4/7/2011 12:35' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.failToParseDateTimeInNewParserError(QueryExecutionErrors.scala:1084)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:148)
	at org.apache.spark.sql.catalyst.util.DateTimeFormatterHelper$$anonfun$checkParsedDiff$1.applyOrElse(DateTimeFormatterHelper.scala:141)
	at scala.runtime.A

In [ ]:
from pyspark.sql.functions import col